In [ ]:
import json
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import openai
import pickle
from typing import List, Dict, Any
import os

API_KEY =os.getenv('OPENAI_API_KEY')
RECIPE_DATA_PATH = "../embedder/recipe_embeddings.pkl"
PRODUCTS_FILE = "../shared_data/biedronka_offers_enhanced.json"

client = openai.OpenAI(api_key=API_KEY)

# Wczytaj przepisy z embeddingami
with open(RECIPE_DATA_PATH, 'rb') as f:
    recipe_data = pickle.load(f)

# Wczytaj produkty z keywords
with open(PRODUCTS_FILE, 'r', encoding='utf-8') as f:
    products_data = json.load(f)

products = products_data['products']

print(f"Załadowano {len(recipe_data['recipes_df'])} przepisów")
print(f"Załadowano {len(products)} produktów z Biedronki")

def search_recipes_by_keywords(keywords: List[str], top_k: int = 5) -> List[Dict]:
    """
    Wyszukuje przepisy używając listy angielskich keywords
    """
    if not keywords:
        return []
    
    try:
        # Tworzymy zapytanie z keywords
        query = " ".join(keywords) + " recipes"
        
        # Get query embedding
        query_response = client.embeddings.create(
            input=[query],
            model='text-embedding-3-small'
        )
        query_embedding = query_response.data[0].embedding
        
        # Calculate similarities
        recipe_embeddings = np.array(recipe_data['embeddings'])
        similarities = cosine_similarity([query_embedding], recipe_embeddings)[0]
        
        # Get top results
        top_indices = np.argsort(similarities)[::-1][:top_k]
        
        results = []
        for idx in top_indices:
            recipe = recipe_data['recipes_df'][idx]
            results.append({
                'title': recipe['Title'],
                'similarity': similarities[idx],
                'ingredients': str(recipe['Cleaned_Ingredients']),
                'recipe_idx': idx
            })
        
        return results
        
    except Exception as e:
        print(f"Błąd wyszukiwania przepisów: {e}")
        return []


def generate_meal_plan_from_products(selected_products: List[Dict], 
                                   days: int = 3, people: int = 2) -> Dict:
    """
    Generuje plan posiłków z konkretnej listy produktów
    """
    print(f"Generowanie planu na {days} dni dla {people} osób...")
    print(f"Używane produkty: {len(selected_products)}")
    
    # Przygotuj kontekst dla LLM - znajdź przepisy dla wybranych produktów
    context = f"DOSTĘPNE PRODUKTY PROMOCYJNE:\n"
    
    for product in selected_products:
        context += f"- {product['name']}: {product['price']} PLN ({product.get('discount_info', 'brak zniżki')})\n"
        
        # Dodaj najlepszy przepis jako inspirację
        keywords = product.get('english_keywords', [])
        if keywords:
            recipes = search_recipes_by_keywords(keywords, top_k=1)
            if recipes:
                best_recipe = recipes[0]
                context += f"  Sugerowany przepis: {best_recipe['title']}\n"
                context += f"  Składniki: {best_recipe['ingredients'][:120]}...\n"
    
    # Prompt dla LLM
    prompt = f"""Stwórz plan posiłków na {days} dni dla {people} osób, wykorzystując produkty promocyjne z Biedronki.

{context}

ZASADY:
- Użyj jak najwięcej produktów promocyjnych z listy
- Stwórz różnorodne, zdrowe posiłki (śniadanie, lunch, kolacja)
- Wykorzystaj sugerowane przepisy jako inspirację
- Podaj oszacowany całkowity koszt
- Możesz dodać podstawowe składniki (chleb, jajka, mleko, etc.)

Zwróć odpowiedź w formacie JSON:
{{
  "plan_info": {{
    "days": {days},
    "people": {people},
    "estimated_total_cost": "XX.XX PLN"
  }},
  "meals": [
    {{
      "day": 1,
      "type": "breakfast",
      "name": "Nazwa posiłku",
      "main_products": ["Produkty z listy promocyjnej"],
      "additional_ingredients": ["Podstawowe składniki jeśli potrzeba"],
      "prep_time": "XX min"
    }}
  ],
  "shopping_summary": {{
    "promotional_products_cost": "XX.XX PLN",
    "additional_ingredients_cost": "XX.XX PLN",
    "total_savings": "Ile zaoszczędzono dzięki promocjom"
  }}
}}"""
    
    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "Jesteś ekspertem od planowania posiłków. Odpowiadasz TYLKO w formacie JSON."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.3,
            max_tokens=2000
        )
        
        result = response.choices[0].message.content.strip()
        
        # Clean JSON response
        if result.startswith('```json'):
            result = result.replace('```json', '').replace('```', '').strip()
        
        parsed_plan = json.loads(result)
        return parsed_plan
        
    except Exception as e:
        print(f"Błąd generowania planu: {e}")
        return None


def quick_meal_plan(product_names: List[str], days: int = 2, people: int = 2):
    """
    Szybkie tworzenie planu z wybranych produktów (po nazwach)
    """
    selected_products = []
    
    for name in product_names:
        product = next((p for p in products if name.lower() in p['name'].lower()), None)
        if product:
            selected_products.append(product)
            print(f"✅ Dodano: {product['name']}")
        else:
            print(f"❌ Nie znaleziono: {name}")
    
    if not selected_products:
        print("❌ Nie wybrano żadnych produktów!")
        return None
    
    plan = generate_meal_plan_from_products(selected_products, days, people)
    
    if plan:
        print(f"\n✅ WYGENEROWANY PLAN:")
        print(json.dumps(plan, ensure_ascii=False, indent=2))
        
        # Zapisz do pliku
        filename = f"meal_plan_{days}days_{people}people.json"
        with open(filename, 'w', encoding='utf-8') as f:
            json.dump(plan, f, ensure_ascii=False, indent=2)
        print(f"\n💾 Plan zapisany do: {filename}")
    
    return plan



Załadowano 300 przepisów
Załadowano 28 produktów z Biedronki


In [2]:
# Pobierz nazwy wszystkich produktów
all_product_names = [product['name'] for product in products]

print(f"\n📦 Produkty do wykorzystania:")
for i, name in enumerate(all_product_names, 1):
    print(f"{i:2d}. {name}")

# Wygeneruj plan ze wszystkich produktów
plan = quick_meal_plan(all_product_names, days=1, people=1)



📦 Produkty do wykorzystania:
 1. Kiełbasa Podwawelska Kraina Wędlin
 2. Szynka Kraina Wędlin, 100 g
 3. Boczek Kraina Wędlin, 100 g
 4. Kiełbasa krakowska parzona w plastrach Kraina Wędlin, 250 g
 5. Kabanosy z wędzarni Kraina Wędlin Select, 175 g
 6. Kiełbasa krakowska z szynki Tarczyński, 330 g
 7. Jogurt proteinowy pitny YoPRO, 270 g
 8. Krem do smarowania Nutella, 350 g
 9. Pasztet Podlaski Drosed, 195 g
10. Gulasz angielski Krakus, 300 g
11. Herbata Lipton Yellow Label, 25 torebek
12. Herbata Teekanne, 20 torebek
13. Kawa ziarnista Jacobs Barista Editions, 1 kg
14. Kapsułki Dolce Gusto, 16 szt.
15. Jajko Kinder Joy, 20 g
16. Olej rzepakowy Kujawski, 2 l
17. Mleko UHT 3,2% Mleczna Dolina, 1 l
18. Ser plastry Światowid, 2x250 g/500 g
19. Odżywcze mleczko lub balsam do ciała Nivea, 400 ml
20. Łopatka wieprzowa
21. Świeże udo lub podudzie kurczaka pakowane próżniowo Kraina Mięs
22. Krewetki gotowane Marinero, 250 g
23. Sok jabłkowy Riviva, 1 l
24. Lekki krem do twarzy NIVEA Care 5w1,